## Cats vs Dogs

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image
import matplotlib as mpl
import math
import image_tools as lit
from sklearn.neighbors import KNeighborsClassifier
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from pathlib import Path
from tqdm.notebook import tqdm
from PIL import Image as Img
import cv2

from __future__ import absolute_import, division, print_function, unicode_literals
import importlib
import ssp19ai_utils.utils as utils
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.base import clone

importlib.reload(utils)
%matplotlib inline


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.pipeline import Pipeline

import ssp19ai_utils.utils as utils
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.multiclass import OneVsOneClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from sklearn.base import clone
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
importlib.reload(utils)

%matplotlib inline


In [3]:
cwd = os.getcwd()
cwd
if not os.path.exists(cwd+"/data/fixed_petImages"):
    os.mkdir(cwd+'/data/fixed_petImages')
if not os.path.exists(cwd+"/data/fixed_petImages/train"):
    os.mkdir(cwd+'/data/fixed_petImages/train')
if not os.path.exists(cwd+"/data/fixed_petImages/test"):
    os.mkdir(cwd+'/data/fixed_petImages/test')

Training a Binary Classifier

In [4]:
def get_from_dir(path_dir):
    filenames = os.listdir('./data/PetImages/'+path_dir+'/')
    categories = []
    for filename in filenames:
        category = filename.split('.')[0]
        if category == 'dog':
            categories.append(1)
        else:
            categories.append(0)
    return pd.DataFrame({
        'filename': filenames,
        'category': categories,
        'data': ''
    })

In [5]:
# resize with white background instead of missing pixels
def resize_with_white_background(path_ori, path_dest, index, path_dir, df):
    try:
        img = cv2.imread(path_ori)
        # resize + color to gray image
        img1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img1 = cv2.resize(img1, (28,28))
        df.at[index,'data'] = img1.reshape(-1, 784)[0]

        path = './data/fixed_petImages/'+path_dir+'/'+path_dest
        cv2.imwrite(path, img1)
    except:
        print("bad file", path_dest)
        return np.array([])
# resize all the images in the 'ori_dir'


In [6]:
def get_data_from_dir(path_dir, df):
    count = 0
    for item in df['filename']:
        file = './data/PetImages/'+path_dir + '/' + item
        resize_with_white_background(file, item, count ,path_dir, df)
        count = count + 1


In [7]:
df = get_from_dir('train')

In [8]:
df

,filename,category,data
0,cat.0.jpg,0,
1,cat.1.jpg,0,
2,cat.10.jpg,0,
3,cat.100.jpg,0,
4,cat.1000.jpg,0,
...,...,...,...
24995,dog.9995.jpg,1,
24996,dog.9996.jpg,1,
24997,dog.9997.jpg,1,
24998,dog.9998.jpg,1,


In [9]:
import matplotlib.image as mpimg

# Create figure with a specified number of subplots
def plot_images(images, labels, sp=3):
  fig, axes = plt.subplots(sp,sp)
  fig.subplots_adjust(hspace=1, wspace=0.3)

  for i, ax in enumerate(axes.flat):
    # Plot image
    ax.imshow(mpimg.imread(images[i]))
	
    # Plot label
    ax.set_xlabel('Label : %s' % labels[i])

  plt.show()

In [10]:
get_data_from_dir('train', df)

In [11]:
X = df['data'].copy().to_numpy()
y = df['category'].copy().to_numpy()


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42)

In [13]:
y_test

array([0, 1, 0, ..., 1, 1, 0], dtype=int64)

In [14]:
X_train

array([array([ 49,  49,  35,  25,  31,  75, 121, 124, 113,  95, 128, 134, 108,
               73,  81,  85,  89, 120, 177, 185, 175, 181, 201, 198, 154, 176,
              192, 198,  49,  48,  32,  26,  34,  86, 128, 124, 111,  34, 118,
              132,  99,  73,  84,  87,  89, 127, 180, 186, 178, 186,  28,  26,
               23, 180, 193, 196,  51,  47,  30,  28,  32, 133, 160, 119, 107,
               50,  48, 127,  88,  75,  83,  89,  91, 135, 183, 184, 175, 109,
               37,  65,  34, 170, 192, 197,  46,  42,  28,  26,  38, 168, 177,
              117, 102,  58,  63,  74,  82,  80,  84,  90,  94, 141, 185, 181,
              169,  35,  58,  79,  79,  40, 193, 197,  51,  41,  25,  29,  37,
              188, 190, 112, 102,  68,  87,  61,  78,  77,  86,  91,  92, 147,
              187, 178,  15,  46,  73,  93,  98,  27, 192, 197,  52,  37,  28,
               30,  42, 179, 184, 109,  99,  79,  72,  54,  69,  83,  87,  90,
               97, 151, 187, 141,  47,  79,  85,  85

In [15]:
X_temp = X_train[:5]
X_temp.shape

(5,)

In [16]:
distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
K = range(1, 10)

for k in K:
    print(k)
    # Building and fitting the model
    kmeanModel = KMeans(n_clusters=k).fit(X_temp)
    kmeanModel.fit(X_temp)

    distortions.append(sum(np.min(cdist(X_temp, kmeanModel.cluster_centers_,
                                        'euclidean'), axis=1)) / X_temp.shape[0])
    inertias.append(kmeanModel.inertia_)

    mapping1[k] = sum(np.min(cdist(X_temp, kmeanModel.cluster_centers_,
                                   'euclidean'), axis=1)) / X_temp.shape[0]
    mapping2[k] = kmeanModel.inertia_

1


In [16]:
knn = KNeighborsClassifier()
log_reg = LogisticRegression(max_iter=100000, random_state=42)
randomForest = RandomForestClassifier()
sgd_clf = SGDClassifier(random_state=42)


In [ ]:
from sklearn import metrics

k_range = range(1,4)
scores = []
for k in k_range:
    print(k)
    knn_ = KNeighborsClassifier(n_neighbors=k)
    knn_.fit(X_train, y_train)
    y_pred = knn_.predict(X_test)
    scores.append(metrics.accuracy_score(y_test, y_pred))

plt.figure(figsize=(12, 6))
plt.plot(k_range, scores,color='red', linestyle='dashed', marker='o',
         markerfacecolor='blue', markersize=10)
plt.xlabel('Value of K for KNN')
plt.ylabel('Testing Accuracy')

# Finding the maximum k - the number of nearest neighbors:
max_score = max(scores)
best_k = scores.index(max_score)
print("The best accuracy of the knn model is when k =",best_k, ", and the score is:",max_score) 

In [ ]:
randomForest_predict = cross_val_predict(
    randomForest, X_train, y_train, cv=5)

In [ ]:
log_reg_predict = cross_val_predict(log_reg, X_train, y_train, cv=5)

In [ ]:
knn_predict = cross_val_predict(knn, X_train, y_train, cv=5)

In [ ]:
SGD_predict = cross_val_predict(sgd_clf, X_train, y_train, cv=5)

Confusion Matrix

In [ ]:
knn_confusion_mat = confusion_matrix(y_train, knn_predict)
log_reg_confusion_mat = confusion_matrix(y_train, log_reg_predict)
randomForest_confusion_mat = confusion_matrix(
    y_train, randomForest_predict)
sgd_clf_confusion_mat = confusion_matrix(y_train, SGD_predict)


Precision and Recall

In [ ]:
KNN_prec = precision_score(y_train, knn_predict, average='weighted')
print(KNN_prec)

KNN_recall = recall_score(y_train, knn_predict, average='weighted')
print(KNN_recall)

KNN_f1 = f1_score(y_train, knn_predict, average='weighted')
print(KNN_f1)

KNN_acc = accuracy_score(y_train, knn_predict)
print(KNN_acc)


In [ ]:
cross_val_score(knn, X_train, y_train, cv=5, scoring="accuracy")


In [ ]:
log_reg_prec = precision_score(y_train, log_reg_predict, average='weighted')
print(log_reg_prec)

log_reg_recall = recall_score(y_train, log_reg_predict, average='weighted')
print(log_reg_recall)

log_reg_f1 = f1_score(y_train, log_reg_predict, average='weighted')
print(log_reg_f1)

log_reg_acc = accuracy_score(y_train, log_reg_predict)
print(log_reg_acc)


In [ ]:
cross_val_score(log_reg, X_train, y_train, cv=5, scoring="accuracy")


In [ ]:
randomForest_prec = precision_score(
    y_train, randomForest_predict, average='weighted')
print(randomForest_prec)

randomForest_recall = recall_score(
    y_train, randomForest_predict, average='weighted')
print(randomForest_recall)

randomForest_f1 = f1_score(
    y_train, randomForest_predict, average='weighted')
print(randomForest_f1)

randomForest_acc = accuracy_score(y_train, randomForest_predict)
print(randomForest_acc)


In [ ]:
cross_val_score(randomForest, X_train, y_train, cv=5, scoring="accuracy")


In [ ]:
SGD_prec = precision_score(y_train, SGD_predict, average='weighted')
print(SGD_prec)

SGD_recall = recall_score(y_train, SGD_predict, average='weighted')
print(SGD_recall)

SGD_f1 = f1_score(y_train, SGD_predict, average='weighted')
print(SGD_f1)

SGD_acc = accuracy_score(y_train, SGD_predict)
print(SGD_acc)


In [ ]:
cross_val_score(sgd_clf, X_train, y_train, cv=5, scoring="accuracy")

# Hard Voting (Ensembele)

In [ ]:
voting_clf = VotingClassifier(
    estimators=[('knn', knn), ('log_reg', log_reg), ('randomForest', randomForest), ('sgd_clf', sgd_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

In [ ]:
max_accuracy_score = 0
final_model = ''
for clf in (knn, log_reg, randomForest, sgd_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy_score_ = accuracy_score(y_test, y_pred)
    if accuracy_score_ > max_accuracy_score:
        final_model = clf
        max_accuracy_score = accuracy_score_
    print(clf.__class__.__name__, accuracy_score_)


# Performance Measures
## Measuring Accuracy Using Cross-Validation

The best model is: 

In [ ]:
final_model 

With accuracy score of:  

In [ ]:
max_accuracy_score

In [ ]:
knn_predict = cross_val_predict(knn, X_train, y_train, cv=5)


In [ ]:
# dTree_predict = cross_val_predict(dTree, X_train, y_train, cv=5)

In [ ]:
randomForest_predict = cross_val_predict(
    randomForest, X_train, y_train, cv=5)

In [ ]:
SGD_predict = cross_val_predict(sgd_clf, X_train, y_train, cv=5)

# Performance Measures
## Measuring Accuracy Using Cross-Validation

In [ ]:
skfolds = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for train_index, test_index in skfolds.split(X_train, y_train_dog):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_dog[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_dog[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))


In [ ]:
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(train_images, train_labels)
ovo_clf.predict([2])